In [142]:
import pandas as pd
import xarray as xr
import numpy as np
from comp_utils import *

In [143]:
# The test_energy_data_20200920_20240519.h5 has been created in the RemitData notebook
test_energy_data = pd.read_hdf('data/combined/test_energy_data_20200920_20240519.h5', 'df')
test_energy_data["dtm"] = pd.to_datetime(test_energy_data["dtm"])
test_energy_data.rename(columns={"dtm": "valid_time"}, inplace=True)
test_energy_data["Wind_MWh_credit"] = 0.5*test_energy_data["Wind_MW"] - test_energy_data["boa_MWh"]
test_energy_data["Solar_MWh_credit"] = 0.5*test_energy_data["Solar_MW"]
test_energy_data["total_generation_MWh"] = test_energy_data["Wind_MWh_credit"] + test_energy_data["Solar_MWh_credit"]

In [144]:
cutoff_reference_time = pd.Timestamp('2024-02-18 23:00:00+00:00')

# Wind
dwd_wind = pd.read_hdf('data/combined/dwd_wind_20200920_20240519.h5', 'df')
dwd_wind = dwd_wind[dwd_wind["valid_time"] - dwd_wind["reference_time"] < np.timedelta64(70, "h")]
dwd_wind = dwd_wind.set_index("valid_time").groupby("reference_time").resample("30min").interpolate("linear")
dwd_wind = dwd_wind.drop(columns="reference_time",axis=1).reset_index()
dwd_wind = dwd_wind[dwd_wind['valid_time'] >= cutoff_reference_time]
print(dwd_wind)

ncep_wind = pd.read_hdf('data/combined/ncep_wind_20200920_20240519.h5', 'df')
ncep_wind = ncep_wind[ncep_wind["valid_time"] - ncep_wind["reference_time"] < np.timedelta64(70, "h")]
ncep_wind = ncep_wind.set_index("valid_time").groupby("reference_time").resample("30min").interpolate("linear")
ncep_wind = ncep_wind.drop(columns="reference_time",axis=1).reset_index()
ncep_wind = ncep_wind[ncep_wind['valid_time'] >= cutoff_reference_time]
print(ncep_wind)

                  reference_time                valid_time  \
683014 2024-02-16 06:00:00+00:00 2024-02-18 23:00:00+00:00   
683015 2024-02-16 06:00:00+00:00 2024-02-18 23:30:00+00:00   
683016 2024-02-16 06:00:00+00:00 2024-02-19 00:00:00+00:00   
683017 2024-02-16 06:00:00+00:00 2024-02-19 00:30:00+00:00   
683018 2024-02-16 06:00:00+00:00 2024-02-19 01:00:00+00:00   
...                          ...                       ...   
734031 2024-05-19 00:00:00+00:00 2024-05-21 19:00:00+00:00   
734032 2024-05-19 00:00:00+00:00 2024-05-21 19:30:00+00:00   
734033 2024-05-19 00:00:00+00:00 2024-05-21 20:00:00+00:00   
734034 2024-05-19 00:00:00+00:00 2024-05-21 20:30:00+00:00   
734035 2024-05-19 00:00:00+00:00 2024-05-21 21:00:00+00:00   

        DWD_WS_W_53.77_1.702  DWD_WS_W_53.77_1.767  DWD_WS_W_53.77_1.832  \
683014              5.175988              4.992588              4.780227   
683015              4.976773              4.824121              4.672352   
683016              4.77755

In [145]:
# Define function to create lag 0 features
def create_lag0_features(df, columns, prefix):
    df = df.sort_values(by=['reference_time', 'valid_time'])
    df[f'{prefix}_avg_lag0'] = df[columns].mean(axis=1)
    df[f'{prefix}_var_lag0'] = df[columns].var(axis=1)
    return df

# Define function to create lagged features with a 30-minute shift
def create_lagged_features(df, columns, prefix):
    df = df.sort_values(by=['reference_time', 'valid_time'])
    for lag in range(-2, 3):  # Including lag +2
        df[f'{prefix}_avg_lag{lag}'] = df[columns].mean(axis=1).shift(lag)
        df[f'{prefix}_var_lag{lag}'] = df[columns].var(axis=1).shift(lag)
    return df

# Create lag 0 features for DWD wind data
dwd_columns_ws = [col for col in dwd_wind.columns if col.startswith('DWD_WS_W_')]
dwd_wind = create_lagged_features(dwd_wind, dwd_columns_ws, 'DWD_WS_W')
dwd_wind = dwd_wind.drop(columns=dwd_columns_ws)

dwd_columns_ws100 = [col for col in dwd_wind.columns if col.startswith('DWD_WS100_W_')]
dwd_wind = create_lagged_features(dwd_wind, dwd_columns_ws100, 'DWD_WS100_W')
dwd_wind = dwd_wind.drop(columns=dwd_columns_ws100)

dwd_columns_wd = [col for col in dwd_wind.columns if col.startswith('DWD_WD_W_')]
dwd_wind = create_lagged_features(dwd_wind, dwd_columns_wd, 'DWD_WD_W')
dwd_wind = dwd_wind.drop(columns=dwd_columns_wd)

dwd_columns_wd100 = [col for col in dwd_wind.columns if col.startswith('DWD_WD100_W_')]
dwd_wind = create_lagged_features(dwd_wind, dwd_columns_wd100, 'DWD_WD100_W')
dwd_wind = dwd_wind.drop(columns=dwd_columns_wd100)

dwd_columns_rh = [col for col in dwd_wind.columns if col.startswith('DWD_RH_W_')]
dwd_wind = create_lag0_features(dwd_wind, dwd_columns_rh, 'DWD_RH_W')
dwd_wind = dwd_wind.drop(columns=dwd_columns_rh)

# Create lag 0 features for NCEP wind data
ncep_columns_ws = [col for col in ncep_wind.columns if col.startswith('NCEP_WS_W_')]
ncep_wind = create_lagged_features(ncep_wind, ncep_columns_ws, 'NCEP_WS_W')
ncep_wind = ncep_wind.drop(columns=ncep_columns_ws)

ncep_columns_ws100 = [col for col in ncep_wind.columns if col.startswith('NCEP_WS100_W_')]
ncep_wind = create_lagged_features(ncep_wind, ncep_columns_ws100, 'NCEP_WS100_W')
ncep_wind = ncep_wind.drop(columns=ncep_columns_ws100)

ncep_columns_wd = [col for col in ncep_wind.columns if col.startswith('NCEP_WD_W_')]
ncep_wind = create_lagged_features(ncep_wind, ncep_columns_wd, 'NCEP_WD_W')
ncep_wind = ncep_wind.drop(columns=ncep_columns_wd)

ncep_columns_wd100 = [col for col in ncep_wind.columns if col.startswith('NCEP_WD100_W_')]
ncep_wind = create_lagged_features(ncep_wind, ncep_columns_wd100, 'NCEP_WD100_W')
ncep_wind = ncep_wind.drop(columns=ncep_columns_wd100)

ncep_columns_rh = [col for col in ncep_wind.columns if col.startswith('NCEP_RH_W_')]
ncep_wind = create_lag0_features(ncep_wind, ncep_columns_rh, 'NCEP_RH_W')
ncep_wind = ncep_wind.drop(columns=ncep_columns_rh)

# Create lagged features for remaining DWD wind data
dwd_columns_remaining = [col for col in dwd_wind.columns if col.startswith('DWD_T_W_')]
dwd_wind = create_lag0_features(dwd_wind, dwd_columns_remaining, 'DWD_T_W')
dwd_wind = dwd_wind.drop(columns=dwd_columns_remaining)

# Create lagged features for remaining NCEP wind data
ncep_columns_remaining = [col for col in ncep_wind.columns if col.startswith('NCEP_T_W_')]
ncep_wind = create_lag0_features(ncep_wind, ncep_columns_remaining, 'NCEP_T_W')
ncep_wind = ncep_wind.drop(columns=ncep_columns_remaining)

# Reset index after modifications
dwd_wind.reset_index(drop=True, inplace=True)
ncep_wind.reset_index(drop=True, inplace=True)

print('DWD Wind:', dwd_wind.head())
print('NCEP Wind:', ncep_wind.head())

DWD Wind:              reference_time                valid_time  DWD_WS_W_avg_lag-2  \
0 2024-02-16 06:00:00+00:00 2024-02-18 23:00:00+00:00            4.376066   
1 2024-02-16 06:00:00+00:00 2024-02-18 23:30:00+00:00            4.667012   
2 2024-02-16 06:00:00+00:00 2024-02-19 00:00:00+00:00            4.957959   
3 2024-02-16 06:00:00+00:00 2024-02-19 00:30:00+00:00            4.637478   
4 2024-02-16 06:00:00+00:00 2024-02-19 01:00:00+00:00            4.316996   

   DWD_WS_W_var_lag-2  DWD_WS_W_avg_lag-1  DWD_WS_W_var_lag-1  \
0            0.017451            4.488179            0.019126   
1            0.010985            4.376066            0.017451   
2            0.011250            4.667012            0.010985   
3            0.000695            4.957959            0.011250   
4            0.015224            4.637478            0.000695   

   DWD_WS_W_avg_lag0  DWD_WS_W_var_lag0  DWD_WS_W_avg_lag1  DWD_WS_W_var_lag1  \
0           4.600292           0.032750                

In [146]:
from datetime import datetime, timedelta
import pandas as pd
import pytz

def find_missing_reference_times(dataset):
    utc = pytz.UTC
    start_date = utc.localize(datetime(2024, 2, 18, 0, 0, 0))
    end_date = utc.localize(datetime(2024, 5, 20, 18, 0, 0))

    # Create a complete set of reference times (00:00, 06:00, 12:00, 18:00 each day) to help us identify missing ref times
    time_periods = [timedelta(hours=6 * i) for i in range(4)]
    all_reference_times = {start_date + timedelta(days=d) + tp for d in range((end_date - start_date).days + 1) for tp in time_periods}
    
    # Filter dataset to get only reference times at 00:00, 06:00, 12:00, and 18:00
    dataset['reference_time'] = pd.to_datetime(dataset['reference_time'])
    filtered_dataset = dataset[dataset['reference_time'].dt.hour.isin([0, 6, 12, 18])]
    dataset_times = set(filtered_dataset['reference_time'])

    # Find missing times
    missing_times = all_reference_times - dataset_times
    
    return sorted(missing_times)



In [147]:
missing_days_dwd_wind= find_missing_reference_times(dwd_wind)
print(missing_days_dwd_wind)

missing_days_ncep_wind= find_missing_reference_times(ncep_wind)
print(missing_days_ncep_wind)

[datetime.datetime(2024, 5, 3, 0, 0, tzinfo=<UTC>), datetime.datetime(2024, 5, 11, 12, 0, tzinfo=<UTC>), datetime.datetime(2024, 5, 11, 18, 0, tzinfo=<UTC>), datetime.datetime(2024, 5, 12, 0, 0, tzinfo=<UTC>), datetime.datetime(2024, 5, 19, 6, 0, tzinfo=<UTC>), datetime.datetime(2024, 5, 19, 12, 0, tzinfo=<UTC>), datetime.datetime(2024, 5, 19, 18, 0, tzinfo=<UTC>), datetime.datetime(2024, 5, 20, 0, 0, tzinfo=<UTC>), datetime.datetime(2024, 5, 20, 6, 0, tzinfo=<UTC>), datetime.datetime(2024, 5, 20, 12, 0, tzinfo=<UTC>), datetime.datetime(2024, 5, 20, 18, 0, tzinfo=<UTC>)]
[datetime.datetime(2024, 3, 20, 6, 0, tzinfo=<UTC>), datetime.datetime(2024, 3, 22, 0, 0, tzinfo=<UTC>), datetime.datetime(2024, 4, 4, 12, 0, tzinfo=<UTC>), datetime.datetime(2024, 4, 15, 12, 0, tzinfo=<UTC>), datetime.datetime(2024, 4, 15, 18, 0, tzinfo=<UTC>), datetime.datetime(2024, 4, 16, 0, 0, tzinfo=<UTC>), datetime.datetime(2024, 4, 16, 18, 0, tzinfo=<UTC>), datetime.datetime(2024, 4, 17, 18, 0, tzinfo=<UTC>), d

In [148]:
import pandas as pd
from datetime import datetime, timedelta

def get_best_reference_time(date, ref_time_list):
    # First check if 6 is available, if not use 0.
    for hour in [6, 0]:
        ref_time = pd.Timestamp(date.year, date.month, date.day, hour, tz='UTC')
        if ref_time in ref_time_list:
            return ref_time
    
    # If 0 is not available go back to the previous day to check for 18, 12 and 6. 
    # Check previous day's preferred hours
    previous_date = date - pd.Timedelta(days=1)
    for hour in [18, 12, 6]:
        ref_time = pd.Timestamp(previous_date.year, previous_date.month, previous_date.day, hour, tz='UTC')
        if ref_time in ref_time_list:
            return ref_time
    
    return None

def process_dataset(dataset):
    features = pd.DataFrame()
    ref_time_list = set(dataset['reference_time'])

    # Get unique dates from dataset
    unique_dates = pd.to_datetime(dataset['valid_time']).dt.normalize().unique()

    # Iterate through unique dates
    for date in unique_dates:
        best_ref_time = get_best_reference_time(date, ref_time_list)
        if best_ref_time:
            # Get day-ahead market times based on the best reference time
            DA_Market_times = day_ahead_market_times(best_ref_time).tz_convert('UTC')
            
            # Filter to include only data within the day-ahead market times for the best reference time
            relevant_data = dataset[(dataset['reference_time'] == best_ref_time) &
                                    (dataset['valid_time'].isin(DA_Market_times))]

            features = pd.concat([features, relevant_data])

    return features.reset_index(drop=True)

In [149]:
dwd_wind = process_dataset(dwd_wind)
ncep_wind = process_dataset(ncep_wind)

In [150]:
dwd_wind = dwd_wind.drop(columns=['reference_time'])
ncep_wind = ncep_wind.drop(columns=['reference_time'])

In [151]:

dwd_wind = dwd_wind.drop(columns=['index'], errors='ignore').reset_index(drop=True)
ncep_wind = ncep_wind.drop(columns=['index'], errors='ignore').reset_index(drop=True)

print(dwd_wind.shape)
print(ncep_wind.shape)
print(test_energy_data.shape)

dwd_wind = dwd_wind.drop_duplicates('valid_time', keep='first')
ncep_wind = ncep_wind.drop_duplicates('valid_time', keep='first')
test_energy_data = test_energy_data.drop_duplicates('valid_time', keep='first')

(4414, 45)
(4414, 45)
(64224, 15)


In [152]:
merged_df = dwd_wind.merge(ncep_wind, on='valid_time', how='left')
merged_df = merged_df.merge(test_energy_data, on='valid_time', how='left')

cutoff_start = pd.Timestamp('2024-02-19 23:00:00+00:00')
cutoff_end = pd.Timestamp('2024-05-19 21:30:00+00:00')
merged_df = merged_df[(merged_df['valid_time'] <= cutoff_end) & 
                                    (merged_df['valid_time'] >= cutoff_start)]

print(merged_df)

# Print the last 50 rows of the filtered DataFrame to verify
#print(merged_df)
#print(merged_df.describe()) 

                    valid_time  DWD_WS_W_avg_lag-2  DWD_WS_W_var_lag-2  \
48   2024-02-19 23:00:00+00:00            7.047743            0.005191   
49   2024-02-19 23:30:00+00:00            6.985611            0.015626   
50   2024-02-20 00:00:00+00:00            6.923482            0.043293   
51   2024-02-20 00:30:00+00:00            7.184665            0.022166   
52   2024-02-20 01:00:00+00:00            7.445848            0.017278   
...                        ...                 ...                 ...   
4361 2024-05-19 19:30:00+00:00            7.027603            0.048874   
4362 2024-05-19 20:00:00+00:00            7.269785            0.011623   
4363 2024-05-19 20:30:00+00:00            7.202606            0.031994   
4364 2024-05-19 21:00:00+00:00            7.135428            0.071490   
4365 2024-05-19 21:30:00+00:00            6.769392            0.146002   

      DWD_WS_W_avg_lag-1  DWD_WS_W_var_lag-1  DWD_WS_W_avg_lag0  \
48              6.975662            0.022862

In [153]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Drop specified columns from merged_df
merged_df = merged_df.drop(['MIP', 'Solar_MW', 'Solar_capacity_mwp', 'Solar_installedcapacity_mwp', 'Wind_MW', 'SS_Price', 'boa_MWh', 'DA_Price', 'Solar_MWh_credit', 'total_generation_MWh'], axis=1)

In [154]:
print(merged_df.describe())
merged_df.to_hdf('data/reference_time_06/WindTestTable.h5', key='df', mode='w')

       DWD_WS_W_avg_lag-2  DWD_WS_W_var_lag-2  DWD_WS_W_avg_lag-1  \
count         4318.000000         4318.000000         4318.000000   
mean             7.223671            0.147168            7.225526   
std              3.632741            0.383328            3.629632   
min              0.305491            0.000671            0.305491   
25%              4.232453            0.027990            4.232453   
50%              7.176060            0.064448            7.190002   
75%             10.146711            0.144173           10.146711   
max             17.226419           10.406014           17.226419   

       DWD_WS_W_var_lag-1  DWD_WS_W_avg_lag0  DWD_WS_W_var_lag0  \
count         4318.000000        4318.000000        4318.000000   
mean             0.147764           7.228622           0.149340   
std              0.384665           3.626992           0.392127   
min              0.000671           0.305491           0.000671   
25%              0.028004           4.23957